# Human-in-the-Loop (HITL) with LangGraph

This notebook demonstrates how to implement Human-in-the-Loop (HITL) functionality in LangGraph - allowing human intervention during the execution of an AI agent workflow.

## Objective 

This notebook demonstrates how to implement **Human-in-the-Loop (HITL)** functionality within a LangGraph agent system. The primary goal is to create an AI assistant that can pause its execution at critical points, allowing a human operator to review, modify, or abort actions before they're taken.

Specifically, we'll learn how to:

- **Create an interruptible agent workflow**  
  Building a graph where execution can be paused at strategic points (in this case, before tools are called)

- **Implement manual approval processes**  
  Allowing humans to review what the AI plans to do before it happens

- **Modify agent behavior mid-execution**  
  Demonstrating how human operators can change parameters, redirect queries, or adjust the agent's intended actions

- **Resume execution after human intervention**  
  Continuing the flow after review, either with the original plan or with human-modified instructions

- **Maintain state across interruptions**  
  Using checkpointing to preserve the agent's state during pauses

---

This approach is particularly valuable for applications where:

- Accuracy is critical and human oversight adds value  
- AI actions might have real-world consequences  
- You want to gradually build trust in an automated system  
- You need an auditable trail of decision-making with human approval steps  
- You're combining automated efficiency with human judgment  

---

The notebook uses a **weather query example**, where a search tool is used to look up information. The human can intercept the search query before it's executed and modify it if needed, showing how HITL can be implemented in a real-world scenario.


## Package Installation

In [3]:
!!pip install langchain langgraph openai pygraphviz

['Requirement already satisfied: langchain in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (0.3.21)',
 'Requirement already satisfied: langgraph in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (0.3.18)',
 'Requirement already satisfied: openai in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (1.66.3)',
 'Collecting pygraphviz',
 '  Using cached pygraphviz-1.14.tar.gz (106 kB)',
 '  Installing build dependencies: started',
 "  Installing build dependencies: finished with status 'done'",
 '  Getting requirements to build wheel: started',
 "  Getting requirements to build wheel: finished with status 'done'",
 '  Preparing metadata (pyproject.toml): started',
 "  Preparing metadata (pyproject.toml): finished with status 'done'",
 'Requirement already satisfied: langchain-core<1.0.0,>=0.3.45 in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (from langchain) (0.3.47)',
 'Requirement already satisfied: langchain-text-splitters<1.0.0,>=0.3.7 in 

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory =  MemorySaver()

In [9]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage


def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [10]:
tool = TavilySearchResults(max_results=2)

Manual human approval

In [11]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

## 🤖 Agent Class – LangGraph Agent with Tool Use and HITL

The `Agent` class defines a **LangGraph-based conversational agent** that can interact with tools and supports **Human-in-the-Loop (HITL)** interruptions before executing actions.

---

### 🔧 Constructor: `__init__`

- **`model`**: A language model instance (e.g., OpenAI's GPT) that is used to generate responses.
- **`tools`**: A list of tools the agent can use to take actions (e.g., web search, calculator).
- **`system`**: An optional system prompt that can be prepended to every LLM call to guide its behavior.
- **`checkpointer`** *(optional)*: Used to persist the agent’s state across interruptions or multiple runs.

---

### 🧠 Inside the Constructor

- A `StateGraph` is initialized using `AgentState`, which defines how the agent maintains state (mainly the list of messages).

#### Two main nodes are added:
- `"llm"`: Calls the language model to generate the next message.
- `"action"`: Executes any tool calls found in the LLM response.

#### A conditional edge is added:
- After the `"llm"` node runs, the graph checks whether the message contains a tool call (`exists_action`).
- If `True`, it transitions to the `"action"` node.
- If `False`, the graph ends (`END`).

- An edge from `"action"` to `"llm"` creates a loop, allowing for continued conversation.
- The graph is compiled with `interrupt_before=["action"]`, which enables HITL behavior (pausing execution before the action is taken).
- Tools are stored in a dictionary for quick lookup.
- The model is bound to the tools using `.bind_tools()` so it can issue structured tool calls.

---

### 📩 Method: `call_openai`

This method sends messages to the LLM.

- Prepends the system message (if provided) to the current conversation.
- Invokes the model using the current messages.
- Returns the newly generated message wrapped in a dictionary.

---

### 🧪 Method: `exists_action`

This function checks if the most recent message from the LLM contains tool calls.

- Prints the current state (for debugging).
- Returns `True` if tool calls exist, triggering the `"action"` node.
- Otherwise, returns `False`, ending the graph execution.

---

### 🛠️ Method: `take_action`

This function processes any tool calls made by the LLM.

- Iterates over each tool call in the latest message.
- For each tool:
  - Finds the appropriate tool function using its name.
  - Calls the tool with the given arguments.
  - Wraps the tool response in a `ToolMessage`.

- Returns the list of tool messages as the result, which are added back to the conversation history.

---

### 🔁 Agent Loop

This loop of **LLM → action → LLM** allows the agent to carry out multi-step reasoning with tool use.

---


In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [13]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='5163e7c6-2152-4883-8d78-0b824978209f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7lHCh3MnZMn0ZV4ZJzZfrqwU', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELY2KTVkKOQPIV16u2CPKhUyhkd0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-78c70358-8f92-47fb-a38f-9c756a09d2d3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current we


## 🧾 Understanding the Agent Output – Tool Call Example

This output is a snapshot of the **AgentState** after one interaction in a LangGraph-based conversational agent. The agent is designed to use external tools (via tool calls) and supports Human-in-the-Loop (HITL) interruption before action execution.

---

### 🧑‍💬 User Message

HumanMessage(content="Whats the weather in SF?", ...)

- Represents the user’s input.
- `content`: The actual question sent by the user.
- `id`: A unique identifier for tracking this message.
- `additional_kwargs` and `response_metadata`: Empty or default values.

---

### 🤖 AI Message

AIMessage(
    content='',
    additional_kwargs={
        'tool_calls': [
            {
                'id': 'call_...',
                'function': {
                    'arguments': '{"query":"current weather in San Francisco"}',
                    'name': 'tavily_search_results_json'
                },
                'type': 'function'
            }
        ],
        'refusal': None
    },
    ...
)

- `content` is empty: The model chose to respond via a **tool call** instead of natural language.
- `tool_calls`: The model generated a structured function call using the tool `tavily_search_results_json`.
- Arguments passed: `{ "query": "current weather in San Francisco" }`
- `finish_reason: 'tool_calls'`: Indicates the model stopped generation because it needs to call a function/tool.
- `model_name`: GPT-3.5-turbo-0125 was used.
- `response_metadata`: Contains information like token usage and system data.

---

### 🔁 Repeated Message

The same `AIMessage` appears again in the output, likely printed from inside the agent logic (such as `exists_action()` or `take_action()`), confirming the tool call was recognized.

---

### 🟰 Final Output: `()`

This is likely from a print statement or a function returning `None` at the end of execution. It indicates no explicit return value at that point in the code.

---

### 🧠 Summary

- The agent received a question: **"What's the weather in SF?"**
- It detected that a tool call was required to fetch real-time data.
- It generated a **structured function call** to a tool named `tavily_search_results_json`.
- The agent paused for HITL or tool execution before proceeding.
- This is part of the **LLM → Tool → LLM** loop in LangGraph.


In [14]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='5163e7c6-2152-4883-8d78-0b824978209f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7lHCh3MnZMn0ZV4ZJzZfrqwU', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELY2KTVkKOQPIV16u2CPKhUyhkd0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-78c70358-8f92-47fb-a38f-9c756a09d2d3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': 

In [15]:
abot.graph.get_state(thread).next

('action',)

continue after interrupt

In [16]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_7lHCh3MnZMn0ZV4ZJzZfrqwU', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Sunday, March 23, 2025. San Francisco, CA - Weather Forecast\', \'url\': \'https://weathershogun.com/weather/usa/ca/san-francisco/480/march/2025-03-23\', \'content\': "Sunday, March 23, 2025. San Francisco, CA - Weather Forecast\\n☰\\nSan Francisco, CA\\n\\nHome Contact Browse States Privacy Policy Terms and Conditions\\n°F)°C)\\n❮\\nToday Tomorrow Hourly 7 days 30 days March\\n❯\\nSan Francisco, California Weather:\\nBeach Hazards Statement (Potential hazards at beaches, such as dangerous currents, high surf, or unexpected waves.)\\nSunday, March 23, 2025\\nDay 66°\\nNight 48°\\nPrecipitation 0 %\\nWind 8 mph\\nUV Index (0 - 11+) 5\\nMonday\\n\\nHourly\\nToday\\nTomorrow\\n7 days\\n30 days [...] Weather Forecast History\\nLast Year\'s Weather on This Day (M

In [17]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='5163e7c6-2152-4883-8d78-0b824978209f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7lHCh3MnZMn0ZV4ZJzZfrqwU', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELY2KTVkKOQPIV16u2CPKhUyhkd0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-78c70358-8f92-47fb-a38f-9c756a09d2d3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': 

In [18]:
abot.graph.get_state(thread).next

()

In [19]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='05a62050-2bfc-434f-8ffa-439596512369'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XVRtWV5Sq3VB1bRXzlBDYVmF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELZqpMtnb0tExRrXYMPIwbeaVKdA', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cec95f4c-95a4-4632-a6d5-f2ea2c2d145e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weat

Modify State

In [20]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='e14eaebc-09f7-4103-948f-b6c152311032'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELcNc8iigrAwgvr8XDYk5t3Ivcgb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-997d4833-2cc1-4a50-9222-b65913dc7d44-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angel

In [21]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='e14eaebc-09f7-4103-948f-b6c152311032'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELcNc8iigrAwgvr8XDYk5t3Ivcgb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-997d4833-2cc1-4a50-9222-b65913dc7d44-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 

In [23]:
current_values = abot.graph.get_state(thread)
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELcNc8iigrAwgvr8XDYk5t3Ivcgb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-997d4833-2cc1-4a50-9222-b65913dc7d44-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 22, 'total_tokens': 174, 'inp

In [24]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Los Angeles'},
  'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch',
  'type': 'tool_call'}]

In [25]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [26]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='e14eaebc-09f7-4103-948f-b6c152311032'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELcNc8iigrAwgvr8XDYk5t3Ivcgb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-997d4833-2cc1-4a50-9222-b65913dc7d44-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in L

{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f008202-fd43-6818-8002-08b40684248a'}}

In [27]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='e14eaebc-09f7-4103-948f-b6c152311032'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BELcNc8iigrAwgvr8XDYk5t3Ivcgb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-997d4833-2cc1-4a50-9222-b65913dc7d44-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 

In [28]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_z3Xe3w1xhtm2SOVyqY23jfch', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Louisiana\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.4411, \'lon\': -91.0551, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1742759320, \'localtime\': \'2025-03-23 14:48\'}, \'current\': {\'last_updated_epoch\': 1742759100, \'last_updated\': \'2025-03-23 14:45\', \'temp_c\': 12.3, \'temp_f\': 54.1, \'is_day\': 1, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/122.png\', \'code\': 1009}, \'wind_mph\': 13.6, \'wind_kph\': 22.0, \'wind_degree\': 244, \'wind_dir\': \'WSW\', \'pressure_mb\': 1007.0, \'pressure_in\': 29.73, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'hu

This walkthrough demonstrates the power of Human-in-the-Loop in LangGraph, allowing human reviewers to:
1. Interrupt execution at specific points
2. Review what the agent is about to do
3. Modify the agent's actions or queries if needed
4. Decide whether to proceed or abort
5. Continue execution with either original or modified state